In [1]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [3]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [22]:
round = "5"
update_game_schedule(f"game-schedule-round-{round}-2023-ncaa-tournament.csv")

{'game_date': '2023-03-25', 'team_2_id': 'florida-atlantic', 'team_1_id': 'kansas-state', 'game_time': '18:09', 'round_num': '5', 'competition_id': '1'}
{'game_date': '2023-03-25', 'team_2_id': 'connecticut', 'team_1_id': 'gonzaga', 'game_time': '20:49', 'round_num': '5', 'competition_id': '1'}
{'game_date': '2023-03-26', 'team_2_id': 'creighton', 'team_1_id': 'san-diego-state', 'game_time': '14:00', 'round_num': '5', 'competition_id': '1'}
{'game_date': '2023-03-26', 'team_2_id': 'miami-fl', 'team_1_id': 'texas', 'game_time': '16:55', 'round_num': '5', 'competition_id': '1'}
data=[{'game_id': 96, 'game_date': '2023-03-25', 'team_1_id': 'kansas-state', 'team_2_id': 'florida-atlantic', 'game_time': '18:09:00', 'round_num': 5, 'competition_id': 1, 'league2_unique': 'ncaambb', 'league1_unique': None}, {'game_id': 97, 'game_date': '2023-03-25', 'team_1_id': 'gonzaga', 'team_2_id': 'connecticut', 'game_time': '20:49:00', 'round_num': 5, 'competition_id': 1, 'league2_unique': 'ncaambb', 'lea

In [6]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = 1, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"{date}-game-scoring-2023-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [27]:
date = datetime.date.today()
# date = "2023-03-24"
generate_game_scoring_sheet(date=date)

In [15]:
def update_scores_from_csv(current_round: int, competition_id: int = 1, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [28]:
date = datetime.date.today()
current_round=5
# date = "2023-03-17"
update_scores_from_csv(current_round=current_round, game_scoring_csv = f"{date}-game-scoring-2023-ncaa-tournament.csv")


losers
data=[{'team_unique': 'creighton', 'competition_id': 1, 'league_unique': 'ncaambb', 'seed': 6, 'overall_seed': 22, 'region': 'SOUTH', 'round_eliminated': 5, 'round_started': 2, 'team_stats': {'pace': 69.4, 'conference': 'big-east', 'offensive_rating': 109.5, 'assist_percentage': 57.8, 'turnover_percentage': 14.8, 'strength_of_schedule': 9.54, 'free_throw_attempt_rate': 0.281, 'three_point_attempt_rate': 0.42, 'true_shooting_percentage': 0.574, 'effective_field_goal_percentage': 0.543, 'two_point_field_goal_percentage': 0.545, 'three_point_field_goal_percentage': 0.36, 'opp_effective_field_goal_percentage': 0.473}, 'stats_thru': '2023-03-12', 'team_win_loss': {'wins': 21.0, 'losses': 12.0, 'games_played': 33.0, 'conference_wins': 14.0, 'conference_losses': 6.0}}] count=None
losers
data=[{'team_unique': 'texas', 'competition_id': 1, 'league_unique': 'ncaambb', 'seed': 2, 'overall_seed': 6, 'region': 'MIDWEST', 'round_eliminated': 5, 'round_started': 2, 'team_stats': {'pace': 70.4,